In [98]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [99]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [100]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [101]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [102]:
sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

def seed_np(integer):
    np.random.seed(integer)

class ERegressor:
    class Net:
        def __init__(self):
            pass

        def set_input_size(self, input_size):
            self.input_size = input_size
        
        def set_weights(self, weights_and_biases):
            self.w1 = weights_and_biases[0]
            self.w2 = weights_and_biases[1]
            self.b1 = weights_and_biases[2]
            self.b2 = weights_and_biases[3]
        
        def predict(self, input):
            return (self.w2.T @ sigmoid(self.w1 @ input.T + self.b1) + self.b2).reshape(-1)
        
        def __add__(self, other):
            w1 = (self.w1 + other.w1) / 2 + np.random.normal(0, 3, (self.input_size, self.input_size))
            w2 = (self.w2 + other.w2) / 2 + np.random.normal(0, 3, (self.input_size, 1))
            b1 = (self.b1 + other.b1) / 2 + np.random.normal(0, 3, (self.input_size, 1))
            b2 = (self.b2 + other.b2) / 2 + np.random.normal(0, 3, 1)

            return w1, w2, b1, b2
    
    def __init__(self, n = 100, random_state = None):
        self.nets = {}
        self.y_preds = {}
        self.best_net = None
        self.best_result = None
        self.n = n - n % 2
        
        for i in range(n - n % 2):
            self.nets[i] = self.Net()

        if random_state != None:
            seed_np(random_state)
    
    def fit(self, X_train, y_train, epochs = 100):
        self.input_size = X_train.shape[1]

        for key in self.nets.keys():
            self.nets[key].set_input_size(self.input_size)

            self.nets[key].set_weights((
                np.random.uniform(-3, 3, (self.input_size, self.input_size)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, 1))
            )

        for epoch in range(epochs):
            for key, _ in self.nets.items():
                self.y_preds[key] = self.nets[key].predict(X_train)
            
            self.mean_absolute_errors = {}
            
            for key, _ in self.y_preds.items():
                self.mean_absolute_errors[key] = mean_absolute_error(y_train, self.y_preds[key])

            self.sorted_indecies = [key for key, _ in sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])]

            if self.best_result != None:
                if self.best_result[1] > sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0][1]:
                    self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                    print(f"Epoch {epoch}: MAE: {self.best_result[1]}")
            else:        
                self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                print(f"Epoch {epoch}: MAE: {self.best_result[1]}")


            self.best_net = self.sorted_indecies[0]
            
            for i in range(0, self.n // 2, 2):
                self.nets[self.sorted_indecies[self.n // 2 + i]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                self.nets[self.sorted_indecies[self.n // 2 + i + 1]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])

    def predict(self, X_test):
        return self.nets[self.best_net].predict(X_test)

In [103]:
regressor = ERegressor(n = 100, random_state = 42)
regressor.fit(scaled_X_train, y_train, epochs = 1000)

Epoch 0: MAE: 15.84001299310242
Epoch 1: MAE: 7.527820295753187
Epoch 3: MAE: 4.925029384674258
Epoch 7: MAE: 3.4242206497323875
Epoch 35: MAE: 3.312248348272221
Epoch 65: MAE: 3.1676560147925197
Epoch 70: MAE: 3.082059255656698
Epoch 151: MAE: 2.936089156576874
Epoch 359: MAE: 2.904530499787042
Epoch 634: MAE: 2.7407265046466605
Epoch 676: MAE: 2.6587334235964124


In [106]:
y_pred = regressor.predict(scaled_X_test)

In [107]:
y_pred

array([28.28567143, 31.09631271, 29.04365286, 33.09716287, 15.37019921,
       20.63192101, 15.65547006, 34.94404715, 14.82618516, 23.6077323 ,
       25.6636302 , 25.71918686, 34.61056749, 16.27544884, 25.5053893 ,
       32.05333024, 15.45064559, 30.73091585, 14.85482581, 25.98605003,
       21.78998809, 21.65877268, 25.71736893, 23.34083676, 32.13035426,
       22.54832179, 21.55406572, 30.15462987, 29.77898494, 22.63721163,
       15.4206301 , 18.37488417, 15.73142314, 22.11371326, 14.5973163 ,
       33.33992368, 22.79083189, 34.84747029, 25.85861526, 15.23366704])

In [108]:
mean_absolute_error(y_test, y_pred)

2.190027265628705